In [1]:
import csv
import numpy

### Read ETF holding data
I'll be comparing two popular Canadian dividend ETFs by looking at their holdings:

CDZ - [iShares S&P/TSX Canadian Dividend Aristocrats Index ETF](https://www.blackrock.com/ca/individual/en/products/239834/ishares-sptsx-canadian-dividend-aristocrats-index-fund)   
XEI - [iShares S&P/TSX Composite High Dividend Index ETF](https://www.blackrock.com/ca/individual/en/products/239846/ishares-sptsx-equity-income-index-etf)

Read in all the holdings from each ETF and use them to create `index -> name` and `name -> index` mappings.

In [2]:
def get_etf_data(csv_file):
    start_row = 11 # Individual holding data starts here
    for i in range(0, start_row - 1):
        csv_file.next()
    
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    return list(csv_reader)

with open('data/CDZ_holdings.csv', 'rb') as csv_file:
    CDZ_data = get_etf_data(csv_file)
    
with open('data/XEI_holdings.csv', 'rb') as csv_file:
    XEI_data = get_etf_data(csv_file)

def get_all_holdings(etfs):
    """Gets combined ETF holdings and index mappings.
    etfs: a list of etfs
    Returns a tuple of holding tickers, index->name mappings, name->index mappings
    """
    combined_holdings = set()
    for etf in etfs:
        # Note that this is a set comprehension, not a dict comprehension.
        holdings = {row['Ticker'] for row in etf}
        combined_holdings = combined_holdings.union(holdings)

    index_to_name = {i: value for i, value in enumerate(combined_holdings)}
    name_to_index = {value: i for i, value in enumerate(combined_holdings)}
    return combined_holdings, index_to_name, name_to_index

combined_holdings, index_to_name, name_to_index = get_all_holdings([CDZ_data, XEI_data])

Create a vector representation for each ETF, using its holdings:  

$\textrm{XEI} = \begin{bmatrix}4.82\\0\\\vdots\\2.77\\0\end{bmatrix} = \begin{bmatrix}\textrm{BCE INC (4.82%)}\\ \textrm{METRO INC (0%)}\\\vdots\\\textrm{INTER PIPELINE LTD (2.77%)}\\\textrm{TOROMONT INDUSTRIES LTD (0%)}\end{bmatrix}$

In [3]:
def build_holdings_vector(etf, combined_holdings, name_to_index):
    n_holdings = len(combined_holdings)
    holdings = numpy.zeros(shape=(n_holdings))
    for row in etf:
        index = name_to_index[row['Ticker']]
        holdings[index] = row['Weight (%)']
        
    return holdings

CDZ = build_holdings_vector(CDZ_data, combined_holdings, name_to_index)
XEI = build_holdings_vector(XEI_data, combined_holdings, name_to_index)

Use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to determine who similair the two ETFs are by holdings.

$$\textrm{similarity} = \cos(\theta) = \frac{A \cdot B}{\lVert A \rVert \times \lVert B \rVert}$$

Cosine similarity return 1 for complete positive correlation, 0 for completely uncorrelated, and -1 for complete negative correlation.

In [4]:
def cosine_similarity(a, b):
    return a.dot(b) / (numpy.linalg.norm(a) * numpy.linalg.norm(b))

holding_correlation = cosine_similarity(CDZ, XEI)
print "Holding correlation: {}".format(holding_correlation)

Holding correlation: 0.52298900804


Do the same thing the holding sectors

In [5]:
# Find all the sectors between the two ETFs

def get_all_sectors(etfs):
    """Gets combined ETF sectors and index mappings.
    etfs: a list of etfs
    Returns a tuple of sectors, index->name mappings, name->index mappings
    """
    combined_sectors = set()
    for etf in etfs:
        # Note that this is a set comprehension, not a dict comprehension.
        sectors = {row['Sector'] for row in etf}
        combined_sectors = combined_sectors.union(sectors)

    index_to_name = {i: value for i, value in enumerate(combined_sectors)}
    name_to_index = {value: i for i, value in enumerate(combined_sectors)}
    return combined_sectors, index_to_name, name_to_index
        
combined_sectors, index_to_name, name_to_index = get_all_sectors([CDZ_data, XEI_data])

def build_sectors_vector(etf, combined_holdings, name_to_index):
    n_sectors = len(combined_holdings)
    sectors = numpy.zeros(shape=(n_sectors))
    for row in etf:
        index = name_to_index[row['Sector']]
        sectors[index] += float(row['Weight (%)'])
        
    return sectors

CDZ_sector_counts = build_sectors_vector(CDZ_data, combined_holdings, name_to_index)
XEI_sector_counts = build_sectors_vector(XEI_data, combined_holdings, name_to_index)

sector_correlation = cosine_similarity(CDZ_sector_counts, XEI_sector_counts)
print "Sector correlation: {}".format(sector_correlation)

Sector correlation: 0.916196528432


Average the holding and sector correlation to get an idea of how similair the two ETFs are

In [6]:
print "Similarity: {}".format(numpy.mean([holding_correlation, sector_correlation]))

Similarity: 0.719592768236
